**broadcast semantic**: 广播语义，用于不同形状的Tensor进行运算，其将较小的Tensor拉伸成较大形状的Tensor，然后再进行加减乘除运算。如果两个tensor是“可广播的”，其需要满足两个条件：
- 每个tensor的维度至少为１，０维tensor是不行的
- 依次比较两个tensor对应维度时，维度值要么相同，要么有一个为１,要么有一个不存在

In [1]:
import torch

In [2]:
x = torch.empty(5,7,3)
y = torch.empty(5,7,3)
# x和y具有相同的形状

x = torch.empty((0,))
y = torch.empty(2,2)
# x维度为0,不满足要求

x = torch.empty(5,3,4,1)
y = torch.empty(  3,1,1)
# x和y满足可广播条件

x = torch.empty(5,2,4,1)
x = torch.empty(  3,1,1)
# 倒数第３个轴上　２!=3，不满足可广播的要求

如果两个tensor x和y是可广播的，则按如下结果进行计算：

-　如果x和y维度不一样，则维度少的tensor增加维度来补全，使两个tensor维度一致
-　对每一个维度，最终维度大小为x和y在该维度上的较大值

In [3]:
x = torch.empty(5,1,4,1)
y = torch.empty(  3,1,1)
(x+y).size()

torch.Size([5, 3, 4, 1])

In [4]:
x = torch.empty(1)
y = torch.empty(3,1,7)
(x+y).size()

torch.Size([3, 1, 7])

In [5]:
x = torch.empty(5,2,4,1)
y = torch.empty(3,1,1)
(x+y).size()

RuntimeError: The size of tensor a (2) must match the size of tensor b (3) at non-singleton dimension 1

#### in-place语义

在进行in-place操作时，即使两个tensor满足可广播的条件，也不允许in-palce tensor改变维度，因为这涉及到内存空间的问题。

In [6]:
ｘ = torch.empty(5,3,4,1)
y = torch.empty(3,1,1)
(x.add_(y)).size()

torch.Size([5, 3, 4, 1])

In [7]:
x = torch.empty(1,3,1) # 不允许x改变维度
y = torch.empty(3,1,7)
(x.add_(y)).size()

RuntimeError: output with shape [1, 3, 1] doesn't match the broadcast shape [3, 3, 7]

#### 反向传播兼容性问题（Backwards Compatibility）

老版本的pytorch允许两个元素总数相同但形状不一致的两个tensor进行元素级运算，运算时，其相当于把两个tensor先拉成１维向量在运算。但现在使用广播机制后，再出现这种情况会出现warning。

如果两个tensor具有相同的元素数，具有不同的形状，而且又正好是可广播的，pytorch不会给出warining反而会按照广播语义进行tensor的扩展，但这时会对后向传播造成不兼容影响

In [9]:
torch.add(torch.ones(4,1),torch.randn(4))

tensor([[0.7724, 0.3463, 0.3580, 1.9530],
        [0.7724, 0.3463, 0.3580, 1.9530],
        [0.7724, 0.3463, 0.3580, 1.9530],
        [0.7724, 0.3463, 0.3580, 1.9530]])

广播语义会将tensor扩展成torch.Size([4,4]),为了检测这种情况造成的错误，可是设置**torch.utils.backcompat.broadcast_warning.enabled = True**

In [10]:
torch.utils.backcompat.broadcast_warning.enabled = True
torch.add(torch.ones(4,1),torch.ones(4))

tensor([[2., 2., 2., 2.],
        [2., 2., 2., 2.],
        [2., 2., 2., 2.],
        [2., 2., 2., 2.]])

教程中给出了warning,但是这里却没有，为什么？_main__:1: UserWarning: self and other do not have the same shape, but are broadcastable, and have the same number of elements.
Changing behavior in a backwards incompatible manner to broadcasting rather than viewing as 1-dimensional.